In [1]:
import numpy as np
import pandas as pd
from data_processing_helper import *
import warnings
warnings.filterwarnings('ignore')

In [2]:
filepath = '/Users/yunyihuang/Desktop/georgelab/gl_local/data/BSB273EC13HSOXYLGA14_output.xlsx'
df = cleanup(filepath)
df.head()

/Users/yunyihuang/Desktop/georgelab/gl_local/data/BSB273EC13HSOXYLGA14_output.xlsx
CLEANING COMPLETED


,Subject,Start Datetime,End Datetime,Filename,Experiment,Group,Box,MSN,FR,Active Lever Presses,...,Timeout Press 111,Timeout Press 112,Timeout Press 113,Timeout Press 114,Timeout Press 115,Timeout Press 116,Timeout Press 117,Timeout Press 118,Timeout Press 119,Timeout Press 120
0,M1369,2021-07-02 18:54:07,2021-07-03 06:54:49,C:\MED-PC IV\DATA\BSB273EC13HSOXYLGA14,0,0,1,OXYGWAS 12h,0,237,...,37644.0,37649.0,37651.0,37657.0,37657.0,37664.0,37665.0,37675.0,37678.0,37758.0
1,M1370,2021-07-02 18:54:11,2021-07-03 06:54:49,BSB273EC13HSOXYLGA14\n,0,0,2,OXYGWAS 12h,0,33,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,M1371,2021-07-02 18:54:14,2021-07-03 06:54:50,BSB273EC13HSOXYLGA14\n,0,0,3,OXYGWAS 12h,0,68,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,M1372,2021-07-02 18:54:18,2021-07-03 06:54:50,BSB273EC13HSOXYLGA14\n,0,0,4,OXYGWAS 12h,0,36,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,M1373,2021-07-02 18:54:20,2021-07-03 06:54:50,BSB273EC13HSOXYLGA14\n,0,0,5,OXYGWAS 12h,0,74,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [3]:
filtered_cols = ['Subject'] + [col for col in df.columns if 'Reward ' in col]
df_reward = df[filtered_cols].sort_values('Subject').reset_index().drop('index',axis=1)
df_reward['allRewards'] = df_reward.iloc[:,1:].values.tolist()
df_reward.head()

,Subject,Reward 1,Reward 2,Reward 3,Reward 4,Reward 5,Reward 6,Reward 7,Reward 8,Reward 9,...,Reward 109,Reward 110,Reward 111,Reward 112,Reward 113,Reward 114,Reward 115,Reward 116,Reward 117,allRewards
0,M1369,18,116,306,373,449,881,1539,1584,2121,...,35025.0,36527.0,36588.0,36842.0,37639.0,37664.0,37757.0,37915.0,42938.0,"[18.0, 116.0, 306.0, 373.0, 449.0, 881.0, 1539..."
1,M1370,425,1430,1584,4192,4218,6212,8386,9683,9713,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[425.0, 1430.0, 1584.0, 4192.0, 4218.0, 6212.0..."
2,M1371,3596,5196,6252,6479,6534,7614,7926,7949,7987,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[3596.0, 5196.0, 6252.0, 6479.0, 6534.0, 7614...."
3,M1372,450,822,2265,3695,5499,6967,8779,8803,11097,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[450.0, 822.0, 2265.0, 3695.0, 5499.0, 6967.0,..."
4,M1373,1,298,901,1716,1821,2716,3027,3399,3894,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[1.0, 298.0, 901.0, 1716.0, 1821.0, 2716.0, 30..."


In [4]:
df_reward['cleanedRewards'] = df_reward['allRewards'].apply(lambda lst: [lst[0]] + [val for val in lst[1:] if val > 0])
df_burst = df_reward[['Subject', 'cleanedRewards']]
df_burst['totalRewards'] = df_burst['cleanedRewards'].apply(lambda x: len(x))
df_burst.head()

,Subject,cleanedRewards,totalRewards
0,M1369,"[18.0, 116.0, 306.0, 373.0, 449.0, 881.0, 1539...",117
1,M1370,"[425.0, 1430.0, 1584.0, 4192.0, 4218.0, 6212.0...",28
2,M1371,"[3596.0, 5196.0, 6252.0, 6479.0, 6534.0, 7614....",57
3,M1372,"[450.0, 822.0, 2265.0, 3695.0, 5499.0, 6967.0,...",25
4,M1373,"[1.0, 298.0, 901.0, 1716.0, 1821.0, 2716.0, 30...",60


In [271]:
def get_bursts(lst):
    interval = 120
    allBursts = []

    i = 0
    while i < len(lst):
        oneBurst = []
        limit = lst[i] + interval
        j = i
        while j < len(lst) and lst[j] <= limit:
            oneBurst.append(lst[j])
            j += 1
        allBursts.append(oneBurst)
        i = j
        
    return allBursts

df_burst['rawBurst'] = df_burst['cleanedRewards'].apply(get_bursts)
df_burst['numBurst'] = df_burst['rawBurst'].apply(lambda x: len([i for i in x if len(i)>1]))

In [272]:
def get_mean_num_rewards(lst):
    bursts = [i for i in lst if len(i) > 1]
    total_rewards = len(list(itertools.chain.from_iterable(bursts)))
    total_bursts = len(bursts)
    
    if total_bursts == 0:
        return 0
    #print(total_rewards, total_bursts)
    
    return round(total_rewards / total_bursts,2)

df_burst['meanNumRewards'] = df_burst['rawBurst'].apply(get_mean_num_rewards)

In [273]:
def get_burst_rewards_pct(lst):
    numRewards = len(list(itertools.chain.from_iterable(lst)))
    bursts = [i for i in lst if len(i) > 1]
    numBurstRewards = len(list(itertools.chain.from_iterable(bursts)))
    return round(numBurstRewards/numRewards,2)

df_burst['pctRewards'] = df_burst['rawBurst'].apply(get_burst_rewards_pct)

In [274]:
# get the maximum number of rewards contain in a single burst in one session
def get_max_burst(lst):
    bursts = [i for i in lst if len(i) > 1]
    if len(bursts) == 0:
        return 0
    return max([len(i) for i in bursts])

df_burst['maxBurst'] = df_burst['rawBurst'].apply(get_max_burst)

In [275]:
df_burst

,Subject,cleanedRewards,totalRewards,rawBurst,numBurst,meanNumRewards,pctRewards,maxBurst
0,M1369,"[18.0, 116.0, 306.0, 373.0, 449.0, 881.0, 1539...",117,"[[18.0, 116.0], [306.0, 373.0], [449.0], [881....",38,2.39,0.78,5
1,M1370,"[425.0, 1430.0, 1584.0, 4192.0, 4218.0, 6212.0...",28,"[[425.0], [1430.0], [1584.0], [4192.0, 4218.0]...",8,2.00,0.57,2
2,M1371,"[3596.0, 5196.0, 6252.0, 6479.0, 6534.0, 7614....",57,"[[3596.0], [5196.0], [6252.0], [6479.0, 6534.0...",7,2.29,0.28,3
3,M1372,"[450.0, 822.0, 2265.0, 3695.0, 5499.0, 6967.0,...",25,"[[450.0], [822.0], [2265.0], [3695.0], [5499.0...",7,2.14,0.60,3
4,M1373,"[1.0, 298.0, 901.0, 1716.0, 1821.0, 2716.0, 30...",60,"[[1.0], [298.0], [901.0], [1716.0, 1821.0], [2...",5,2.20,0.18,3
5,M1374,"[369.0, 407.0, 877.0, 1348.0, 2010.0, 2064.0, ...",38,"[[369.0, 407.0], [877.0], [1348.0], [2010.0, 2...",6,3.50,0.55,5
6,M1375,"[12.0, 191.0, 871.0, 1651.0, 2221.0, 3012.0, 3...",53,"[[12.0], [191.0], [871.0], [1651.0], [2221.0],...",0,0.00,0.00,0
7,M1376,"[43.0, 588.0, 1248.0, 1393.0, 1447.0, 7898.0, ...",12,"[[43.0], [588.0], [1248.0], [1393.0, 1447.0], ...",2,2.00,0.33,2
8,M1377,"[1351.0, 2171.0, 2194.0, 2734.0, 2763.0, 3417....",98,"[[1351.0], [2171.0, 2194.0], [2734.0, 2763.0],...",30,2.57,0.79,4
9,M1378,"[47.0, 118.0, 307.0, 393.0, 1184.0, 2201.0, 26...",55,"[[47.0, 118.0], [307.0, 393.0], [1184.0], [220...",10,2.10,0.38,3


In [277]:
df_burst['rawBurst'][6]

[[12.0],
 [191.0],
 [871.0],
 [1651.0],
 [2221.0],
 [3012.0],
 [3574.0],
 [4197.0],
 [4881.0],
 [5658.0],
 [6659.0],
 [7453.0],
 [8099.0],
 [8927.0],
 [9832.0],
 [10478.0],
 [11256.0],
 [12066.0],
 [12602.0],
 [13182.0],
 [13850.0],
 [14517.0],
 [15437.0],
 [15994.0],
 [16591.0],
 [16928.0],
 [17453.0],
 [18071.0],
 [18860.0],
 [19503.0],
 [20198.0],
 [21093.0],
 [25067.0],
 [25737.0],
 [26149.0],
 [26778.0],
 [27364.0],
 [28036.0],
 [28868.0],
 [29721.0],
 [30286.0],
 [30834.0],
 [31542.0],
 [32281.0],
 [33029.0],
 [33868.0],
 [37677.0],
 [38179.0],
 [38714.0],
 [39229.0],
 [39925.0],
 [40600.0],
 [41839.0]]